<h2>Convert all xml to csv</h2>
<p>Traverse through the files can change them to csv</p>

In [1]:
# -*- coding: utf-8 -*-
#Import packages
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import timeit
import codecs
from bs4 import BeautifulSoup # $ pip install beautifulsoup4

In [2]:
#Search through the dataset directory to get all of the file directory.
mypath = 'D:\Downloads\\texts_tagged_200'
from os import walk
files = []
fileName = []
d = [mypath]
for (dirpath, dirnames, filenames) in walk(mypath):
    if(dirpath)!='./': dirpath+="/"
    files.extend([dirpath+s for s in filenames])
    fileName.extend([s for s in filenames])


In [3]:
print(files[0]) #sample
print(len(files))

D:\Downloads\texts_tagged_200\1morenews.com/1.xml
162516


<h3>First step</h3>
<p>We will create a function that accept file directory, then read it and find any word and form them as sentence</p>

In [4]:
def xml2text(directory):
    file = codecs.open(directory,'r','utf-8')
    fileString = file.read()
    soup = BeautifulSoup(fileString,'lxml')
    sentences = soup.find_all("se")
    all_sentences = []
    for sentence in sentences:
        words = sentence.findAll('w')
        wordsList = [word.getText() for word in words]
        if(len(wordsList)>3): all_sentences.append(wordsList)
    return all_sentences
xml2text(files[5])

[['รอง', 'โฆษก', 'ประจำ', 'สำนัก', 'นายก', 'รัฐมนตรี'],
 ['ให้', 'สัมภาษณ์', 'ผ่าน', 'เฟซบุ๊ก', 'ไลฟ์'],
 ['บี', 'บี', 'ซี', 'ไทย'],
 ['ใน',
  'ประเด็น',
  'สถานการณ์',
  'สิทธิ',
  'มนุษย',
  'ชน',
  'ใน',
  'วาระ',
  'ครบ',
  'รอบ'],
 ['ปี', 'รัฐประหาร', 'ของ', 'คณะ', 'รักษา', 'ความ', 'สงบ', 'แห่ง', 'ชาติ'],
 ['เป็น', 'เรื่อง', 'ที่', 'มี', 'ความ', 'สำคัญ', 'เปราะบาง'],
 ['แต่',
  'ขอ',
  'อย่า',
  'เอา',
  'ภาพ',
  'รวม',
  'หรือ',
  'มาตรฐาน',
  'ของ',
  'ที่',
  'ใด',
  'ที่',
  'หนึ่ง',
  'มา',
  'ใช้'],
 ['ซึ่ง',
  'สิทธิ',
  'มนุษย',
  'ชน',
  'กับ',
  'กฎหมาย',
  'ค่อน',
  'ข้าง',
  'ที่',
  'จะ',
  'ใกล้',
  'เคียง',
  'กัน'],
 ['เพียง', 'แต่', 'มี', 'เส้น', 'บางๆ'],
 ['ซึ่ง',
  'ตน',
  'เข้าใจ',
  'บทบาท',
  'ของ',
  'นัก',
  'สิทธิ',
  'มนุษ',
  'ชน',
  'และ',
  'สื่อมวลชน'],
 ['แต่',
  'ก็',
  'ขอ',
  'ให้',
  'ทั้ง',
  'สื่อ',
  'และ',
  'นัก',
  'สิทธิ',
  'มนุษย',
  'ชน',
  'เข้าใจ',
  'ความ',
  'เปราะบาง',
  'ตรง',
  'นี้',
  'ด้วย'],
 ['เพราะ',
  'บาง',
  'กรณี',
  'ห

<h3>Apply this to all of the files.</h3>

In [5]:
sentences = []
start = timeit.default_timer()
for file in files:
    all_sentences = xml2text(file)
    sentences.extend(all_sentences)
stop = timeit.default_timer()
print("Execution time : "+str(round(stop - start,2))+"s.")

Execution time : 10674.41s.


In [ ]:
def construct_sentence_df(files):
    sentences = []
    filePath = []
    
    for file in files[0:5]:
        all_sentences = xml2text(file)
        sentences.extend(all_sentences)
        filPath.extend([file]*len(all_sentences))
    
start = timeit.default_timer()
construct_sentence_df(files)
stop = timeit.default_timer()
print("Execution time : "+str(round(stop - start,2))+"s.")

In [6]:
len(sentences)

4513959

In [7]:
sentences[10:20]

[['หรือ', 'แม้', 'แต่', 'ที่', 'ทำ', 'การ', 'สมาคม'],
 ['แม้', 'แต่', 'การ', 'ออก', 'สื่อ', 'ต่าง'],
 ['และ', 'สื่อ', 'อี', 'เลคโทรนิค'],
 ['ข้อ',
  'น่า',
  'เคลือบแคลง',
  'ใจ',
  'ซึ่ง',
  'เป็น',
  'ประเด็น',
  'ใหญ่',
  'และ',
  'เป็น',
  'คำ',
  'ถาม',
  'คา',
  'ใจ'],
 ['การ', 'ขาย', 'เสื้อ', 'กีฬา'],
 ['ชุด', 'ฟุตบอล', 'ทีม', 'ชาติ', 'ไทย'],
 ['ทั้ง', 'เสื้อ', 'ทำ', 'การ', 'แข่งขัน'],
 ['กางเกง', 'ขา', 'สั้น', 'ทำ', 'การ', 'แข่งขัน'],
 ['นัก', 'ฟุตบอล', 'ทีม', 'ชาติ', 'ไทย'],
 ['รวม', 'ไป', 'ถึง', 'เสื้อ', 'กอง', 'เชียร์', 'ทีม', 'ชาติ', 'ไทย']]

<h2>Next is word2vec!</h2>
<p>After we have obtain sentences, we will train it using gensim word2vec</p>

In [33]:
# Import the built-in logging module and configure it so that Word2Vec 
# creates nice output messages
import logging
logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s',\
    level=logging.INFO)

# Set values for various parameters
num_features = 500    # Word vector dimensionality                      
min_word_count = 40   # Minimum word count                        
num_workers = 4       # Number of threads to run in parallel
context = 10          # Context window size                                                                                    
downsampling = 1e-3   # Downsample setting for frequent words

# Initialize and train the model (this will take some time)
from gensim.models import word2vec
print("Training model...")
model = word2vec.Word2Vec(sentences, workers=num_workers, \
            size=num_features, min_count = min_word_count, \
            window = context, sample = downsampling)




2017-04-29 01:44:26,467 : INFO : collecting all words and their counts
2017-04-29 01:44:26,471 : INFO : PROGRESS: at sentence #0, processed 0 words, keeping 0 word types
2017-04-29 01:44:26,541 : INFO : PROGRESS: at sentence #10000, processed 84419 words, keeping 4379 word types
2017-04-29 01:44:26,561 : INFO : PROGRESS: at sentence #20000, processed 167858 words, keeping 6032 word types
2017-04-29 01:44:26,586 : INFO : PROGRESS: at sentence #30000, processed 253369 words, keeping 6866 word types
2017-04-29 01:44:26,611 : INFO : PROGRESS: at sentence #40000, processed 341927 words, keeping 7377 word types
2017-04-29 01:44:26,634 : INFO : PROGRESS: at sentence #50000, processed 418611 words, keeping 8226 word types
2017-04-29 01:44:26,659 : INFO : PROGRESS: at sentence #60000, processed 495286 words, keeping 8809 word types


Training model...


2017-04-29 01:44:26,684 : INFO : PROGRESS: at sentence #70000, processed 584775 words, keeping 10161 word types
2017-04-29 01:44:26,722 : INFO : PROGRESS: at sentence #80000, processed 702555 words, keeping 10190 word types
2017-04-29 01:44:26,745 : INFO : PROGRESS: at sentence #90000, processed 821315 words, keeping 10194 word types
2017-04-29 01:44:26,770 : INFO : PROGRESS: at sentence #100000, processed 939945 words, keeping 10200 word types
2017-04-29 01:44:26,801 : INFO : PROGRESS: at sentence #110000, processed 1057241 words, keeping 10266 word types
2017-04-29 01:44:26,842 : INFO : PROGRESS: at sentence #120000, processed 1176010 words, keeping 10272 word types
2017-04-29 01:44:26,870 : INFO : PROGRESS: at sentence #130000, processed 1294552 words, keeping 10320 word types
2017-04-29 01:44:26,893 : INFO : PROGRESS: at sentence #140000, processed 1413095 words, keeping 10335 word types
2017-04-29 01:44:26,929 : INFO : PROGRESS: at sentence #150000, processed 1531997 words, keepin

MemoryError: 

In [48]:
# If you don't plan to train the model any further, calling 
# init_sims will make the model much more memory-efficient.
model.init_sims(replace=True)

# It can be helpful to create a meaningful model name and 
# save the model for later use. You can load it later using Word2Vec.load()
model_name = "models/thai_tagged_context.h5"
model.save(model_name)

2017-04-29 01:56:26,980 : INFO : precomputing L2-norms of word weight vectors
2017-04-29 01:56:27,080 : INFO : saving Word2Vec object under models/thai_tagged_context.h5, separately None
2017-04-29 01:56:27,161 : INFO : not storing attribute syn0norm
2017-04-29 01:56:27,162 : INFO : not storing attribute cum_table
2017-04-29 01:56:30,641 : INFO : saved models/thai_tagged_context.h5


In [49]:
model.most_similar('นักข่าว')

[('ผู้สื่อข่าว', 0.7021355628967285),
 ('คอลัมนิสต์', 0.5769671201705933),
 ('สื่อมวลชน', 0.5283889770507812),
 ('หนังสือพิมพ์', 0.5068046450614929),
 ('บรรณาธิการ', 0.5045503377914429),
 ('ไทยรัฐ', 0.4857231378555298),
 ('เดลินิวส์', 0.4820653796195984),
 ('นายแบบ', 0.4544511139392853),
 ('นักการเมือง', 0.441914826631546),
 ('โฆษก', 0.43304669857025146)]

In [42]:
sentences[1000]

['เรื่อง', 'นี้', 'ขอ', 'ให้', 'ดู', 'ให้', 'ละเอียด', 'ด้วย']

In [52]:
model.most_similar('สวย')

[('งดงาม', 0.41707637906074524),
 ('วงศ์วาน', 0.40674978494644165),
 ('หมดจด', 0.3905469477176666),
 ('เซ็กซี่', 0.37484011054039),
 ('เฉิดฉาย', 0.3676585853099823),
 ('ดี', 0.36575961112976074),
 ('ฉูดฉาด', 0.3633784353733063),
 ('สวยๆ', 0.36257246136665344),
 ('สง่า', 0.35836389660835266),
 ('สดใส', 0.35804134607315063)]

In [57]:
model.most_similar('คุณ')

[('เธอ', 0.3916650116443634),
 ('พี่', 0.38157570362091064),
 ('ปู่', 0.3776712417602539),
 ('ป้า', 0.34502461552619934),
 ('คุณหญิง', 0.34252119064331055),
 ('พระคุณ', 0.34112977981567383),
 ('แม่', 0.33915263414382935),
 ('ท่าน', 0.33747851848602295),
 ('ลูกๆ', 0.32694211602211),
 ('ขรัว', 0.31382665038108826)]

In [59]:
model.most_similar('แต่')

[('แต่ว่า', 0.5858616232872009),
 ('เพราะ', 0.4237790107727051),
 ('แค่', 0.37721535563468933),
 ('อะไร', 0.35391560196876526),
 ('เป็นไร', 0.34483757615089417),
 ('หาก', 0.33637097477912903),
 ('ถ้า', 0.3362531363964081),
 ('จะ', 0.33621129393577576),
 ('ไม่', 0.320434033870697),
 ('แยแส', 0.3190068304538727)]

In [60]:
model.most_similar('จีบ')

[('นุ่ง', 0.4197158217430115),
 ('ไฮโซ', 0.4026893973350525),
 ('ผ้าถุง', 0.38462865352630615),
 ('ห้าว', 0.3827100694179535),
 ('ง้อ', 0.37864500284194946),
 ('ปากจัด', 0.3751307427883148),
 ('โอ๋', 0.3744509816169739),
 ('กระโปรง', 0.36587175726890564),
 ('เม้าท์', 0.3618544638156891),
 ('เกี้ยว', 0.35925015807151794)]